# 1. Import of input data

In [1]:
import nbimporter
import sys
import os
from pathlib import Path

In [2]:
#Import libraries
try:
    current_dir = RUN_DIR
    print(f"📂 CombinedHeatPower_CombinedCycle.ipynb executed by RUN.ipynb — using directory: {current_dir}")
except NameError:

    current_dir = Path().resolve()
    print(f"📁 CombinedHeatPower_CombinedCycle.ipynb executed standalone — using directory: {current_dir}")

Libraries_path = (current_dir / "Libraries.ipynb").resolve()

# If not found, try in the parent levels
if not Libraries_path.exists():
    Libraries_path = (current_dir.parent / "Libraries.ipynb").resolve()

# Run the file
if Libraries_path.exists():
    print(f"📘 Importing libraries from: {Libraries_path}")
    get_ipython().run_line_magic('run', f'"{Libraries_path}"')
else:
    print("⚠️ Libraries.ipynb not found! Some functions might not be available.")

📁 CombinedHeatPower_CombinedCycle.ipynb executed standalone — using directory: C:\Users\RELab\OneDrive - Politecnico di Milano\grupporelab\02_Progetti\AREA LCA\Kevin\PhD\aa20232024\03_Sviluppo\Nuovo modello\FINALE\bw\Energy_systems
📘 Importing libraries from: C:\Users\RELab\OneDrive - Politecnico di Milano\grupporelab\02_Progetti\AREA LCA\Kevin\PhD\aa20232024\03_Sviluppo\Nuovo modello\FINALE\bw\Libraries.ipynb
📁 Libraries.ipynb — directory impostata a: C:\Users\RELab\OneDrive - Politecnico di Milano\grupporelab\02_Progetti\AREA LCA\Kevin\PhD\aa20232024\03_Sviluppo\Nuovo modello\FINALE\bw


In [3]:
energysystem = pd.read_excel(energysystem_path, sheet_name = 'CHP-Combined Cycle')

# 2. Input parameters

In [4]:
Power = energysystem['Thermal capacity [kW]']
EfficiencyW = energysystem['Electrical efficiency based on LHV (between 0 and 1)']
EfficiencyQ = energysystem['Heat efficiency based on LHV (between 0 and 1)']
ElectricEnergyproduced = energysystem['Electric energy produced in a year (exported and consumed) [kWh /year]']
ThermalEnergyproduced_primary = energysystem['Thermal energy produced used [kWh /year]']
Qused = 1
SupplyTemperature = energysystem['Supply temperature [°C]']
ReturnTemperature = energysystem['Return temperature [°C]']
EnergyInputSourceGasBlend = energysystem['Energy input from gas-blend [kWh/year]']
EnergyInputSourceBiogas = energysystem['Energy input from biogas [kWh/year]']
EnergyInputSourceHydrogen = energysystem['Energy input from hydrogen [kWh/year]']

EquivalentHours = ThermalEnergyproduced_primary / Power

In [5]:
LHV = 36.8  # In MJ per Nm3 - Table 3.6 06_V_Natural Gas 3_3 Fuel Data
HHV = 40.4  # In MJ per Nm3 - Table 3.6 06_V_Natural Gas 3_3 Fuel Data

In [6]:
Lifespan = np.where(
    Power * (EfficiencyW / EfficiencyQ)  < 25,
    40000 / EquivalentHours,
    100000 / EquivalentHours
)

In [7]:
# ADDITIONAL CALCULATIONS
Power = np.array(Power)
Lifespan = np.array(Lifespan)
EfficiencyW = np.array(EfficiencyW)
EfficiencyQ = np.array(EfficiencyQ)
EfficiencyTot = EfficiencyW + EfficiencyQ
EnergyProducedLHV = (Power * Lifespan) /EfficiencyW
EnergyProducedHHV = EnergyProducedLHV * (HHV/LHV)
TotW = EnergyProducedLHV * EfficiencyW
TotQ = EnergyProducedLHV * EfficiencyQ * Qused

In [8]:
# MULTIFUNCTIONALITY AND ALLOCATION OF output
ElectricEnergyproduced = (EnergyProducedHHV / EfficiencyQ) * EfficiencyW #mode cogen
InletEnergyCogenmode = (ElectricEnergyproduced) / EfficiencyW 
DeltaElProduced = InletEnergyCogenmode * 0.55 - ElectricEnergyproduced #from BREF EU
if Multifunctionality == 'Default': #Power loss method
    ThermalEnergyproduced = EnergyProducedHHV
    ElectricEnergyproduced = (EnergyProducedHHV / EfficiencyQ) * EfficiencyW
    # AFQ calculation
    AFQ = DeltaElProduced / (ElectricEnergyproduced + DeltaElProduced)
    AFQ_CombinedCycle = np.array(AFQ)
    # AFW calculation
    AFW_CombinedCycle = (1 - AFQ_CombinedCycle)


else: #Energy carrier is the natural gas
    InletEnergyCogenmode = EnergyProducedHHV / EfficiencyQ 
    fwecr = 0.7 #weighting factor from energy carrier. EN 15316: 2017 - table B2 pag. 24
    fweexp = fwecr

    AFQ = (InletEnergyCogenmode * fwecr - (ElectricEnergyproduced *fweexp)) / EnergyProducedHHV
    AFW = (1 - AFQ)
    AFQ_CombinedCycle = np.array(AFQ)
    AFW_CombinedCycle = np.array(AFW)

In [9]:
PowerEle = (Power / EfficiencyQ) * EfficiencyW 
RefPowerEle = 430000 #kW
RefMass = 2.79E+07
mass = PowerEle * (RefMass / RefPowerEle)

# 3. Life cycle inventories

## 3.1 Module A1-3

In [10]:
CombinedCycle_aluminiumCastalloy = 0.0105 * mass * AFQ_CombinedCycle
CombinedCycle_aluminiumWroughtalloy = 0.0222 * mass * AFQ_CombinedCycle
CombinedCycle_ceramictile = 0.00003 * mass * AFQ_CombinedCycle
CombinedCycle_chromium = 0.00007 * mass * AFQ_CombinedCycle
CombinedCycle_cobalt = 0.00005 * mass * AFQ_CombinedCycle
CombinedCycle_concreteAT = 2127 * (PowerEle / RefPowerEle) * AFQ_CombinedCycle
CombinedCycle_concreteCH = 3872 * (PowerEle / RefPowerEle) * AFQ_CombinedCycle
CombinedCycle_copper = 0.0327 * mass * AFQ_CombinedCycle
CombinedCycle_diesel = 3441860 * (PowerEle / RefPowerEle) * AFQ_CombinedCycle
CombinedCycle_electricityA13 = 3020000 * (PowerEle / RefPowerEle) * AFQ_CombinedCycle
CombinedCycle_heat = 140600000 * (PowerEle / RefPowerEle) * AFQ_CombinedCycle
CombinedCycle_nickel = 0.0047 * mass * AFQ_CombinedCycle
CombinedCycle_PEHD = 0.031 * mass * AFQ_CombinedCycle
CombinedCycle_reinforcingsteel = 0.6542 * mass * AFQ_CombinedCycle
CombinedCycle_chromiumsteel = 0.0645 * mass * AFQ_CombinedCycle
CombinedCycle_stonewool = 0.0 * mass * AFQ_CombinedCycle

## 3.2 Module A4

In [11]:
CombinedCycle_Truk_16_32 = (((500 + 200)*(mass)/1000)) * AFQ_CombinedCycle
CombinedCycle_Light_Truk = ((50*(mass)/1000)) * AFQ_CombinedCycle

## 3.3 Module B2

In [12]:
CombinedCycle_LightTruk_B2 = ((50 * (Lifespan/2) * 0.01 * (mass/1000))) * AFQ_CombinedCycle

## 3.4 Module B4

In [13]:
CombinedCycleNumberReplacements = (np.ceil(np.divide(ReqSL, Lifespan) - 1))
SFCombinedCycle = (RSP / ReqSL)  # Scaling factor

## 3.5 Module B6

In [14]:
#Repartition parameters for energy source input
GasBlend = EnergyInputSourceGasBlend / (EnergyInputSourceGasBlend + EnergyInputSourceBiogas + EnergyInputSourceHydrogen)
Biogas = EnergyInputSourceBiogas / (EnergyInputSourceGasBlend + EnergyInputSourceBiogas + EnergyInputSourceHydrogen)
Hydrogen = EnergyInputSourceHydrogen / (EnergyInputSourceGasBlend + EnergyInputSourceBiogas + EnergyInputSourceHydrogen)


In [15]:
x = 1
TotEnergyNeeds = np.array([EnergyProducedHHV])
TotEnergyNeeds = TotEnergyNeeds.flatten()
y = (TotEnergyNeeds * 3.6).flatten() * AFQ_CombinedCycle

CombinedCycle_EnergyVectorNG = (((1 / (HHV)) * 0.95) * A) * x * y * GasBlend
CombinedCycle_EnergyVectorBiomethane = (((1 / (HHV)) * 0.042) * B) * x * y * GasBlend
CombinedCycle_EnergyVectorHydrogen = ((((1 / 0.9) * (1 / 3.6)) * 120) * x * y) * C * GasBlend #120 MJ/kg
CombinedCycle_EnergyVectorInputBiogas = ((1 / (HHV))) * x * y * Biogas
CombinedCycle_EnergyVectorInputHydrogen = ((((1 / 0.9) * (1 / 3.6)) * 120) * x * y) * Hydrogen #120 MJ/kg
CombinedCycle_WaterCooling = (0.00662935643564359 * x * y).flatten() 
CombinedCycle_WaterIn2 = (0.00579207920792081 * x * y).flatten()
CombinedCycle_WaterIn3 = (0.193069306930693 * x * y).flatten()
CombinedCycle_WaterOut4 = (9.65346534653468E-07 * x * y).flatten()
CombinedCycle_Acenaphthene = (0.000000000000765520 * x * y).flatten()
CombinedCycle_Acetaldehyde = (7.72277227722775E-10 * x * y).flatten()
CombinedCycle_AceticAcid = (1.1680693069307E-07 * x * y).flatten()
CombinedCycle_Arsenic = (3.96757425742575E-11 * x * y).flatten()
CombinedCycle_Benzene = (8.70742574257428E-10 * x * y).flatten()
CombinedCycle_Benzoapyrene = (5.10668316831685E-13 * x * y).flatten()
CombinedCycle_Beryllium = (2.38440594059407E-12 * x * y).flatten()
CombinedCycle_Butane = (8.93910891089111E-07 * x * y).flatten()
CombinedCycle_Cadmium = (2.18168316831684E-10 * x * y).flatten()
CombinedCycle_CarbonDioxideFossil = (0.0515495049504952 * x * y).flatten()
CombinedCycle_CarbonMonoxideFossil = (2.12376237623763E-06 * x * y).flatten()
CombinedCycle_Chromium = (2.77054455445546E-10 * x * y).flatten()
CombinedCycle_Cobalt = (1.66039603960396E-11 * x * y).flatten()
CombinedCycle_DinitrogenMonoxide = (9.43143564356438E-07 * x * y).flatten()
CombinedCycle_Ethane = (1.32252475247525E-06 * x * y).flatten()
CombinedCycle_Formaldehyde = (3.1180693069307E-08 * x * y).flatten()
CombinedCycle_Hexane = (0.0000007655198019802 * x * y).flatten()
CombinedCycle_Lead = (9.94306930693072E-11 * x * y).flatten()
CombinedCycle_Manganese = (7.53935643564359E-11 * x * y).flatten()
CombinedCycle_Mercury = (6.64158415841586E-11 * x * y).flatten()
CombinedCycle_MethaneFossil = (9.37351485148517E-07 * x * y).flatten()
CombinedCycle_Nickel = (4.16064356435644E-10 * x * y).flatten()
CombinedCycle_NitrogenOxides = (0.0000246163366336634 * x * y).flatten()
CombinedCycle_PAH = (7.72277227722775E-09 * x * y).flatten()
CombinedCycle_Particulates = (4.73985148514853E-07 * x * y).flatten()
CombinedCycle_Pentane = (1.11014851485149E-06 * x * y).flatten()
CombinedCycle_Propane = (6.80569306930695E-07 * x * y).flatten()
CombinedCycle_PropionicAcid = (1.54455445544554E-08 * x * y).flatten()
CombinedCycle_Selenium = (4.73985148514853E-12 * x * y).flatten()
CombinedCycle_SulfurDioxide = (5.52178217821784E-07 * x * y).flatten()
CombinedCycle_Toluene = (1.4480198019802E-09 * x * y).flatten()
CombinedCycle_WaterAir = (0.0000813864357863667 * x * y).flatten()
CombinedCycle_WaterWater = (0.00674683138599583 * x * y).flatten()

## 3.6 Module C2

In [16]:
CombinedCycle_Transport_to_treatment_plant = (50 * (mass/1000)) * AFQ_CombinedCycle

## 3.7 Module C3

In [17]:
CombinedCycle_EOL_aluminiumCastalloy = CombinedCycle_aluminiumCastalloy
CombinedCycle_EOL_aluminiumWroughtalloy = CombinedCycle_aluminiumWroughtalloy
CombinedCycle_EOL_ceramictile = CombinedCycle_ceramictile
CombinedCycle_EOL_chromium = CombinedCycle_chromium
CombinedCycle_EOL_cobalt = CombinedCycle_cobalt
CombinedCycle_EOL_concreteAT = CombinedCycle_concreteAT
CombinedCycle_EOL_concreteCH = CombinedCycle_concreteCH
CombinedCycle_EOL_copper = CombinedCycle_copper
CombinedCycle_EOL_nickel = CombinedCycle_nickel
CombinedCycle_EOL_PEHD = CombinedCycle_PEHD
CombinedCycle_EOL_reinforcingsteel = CombinedCycle_reinforcingsteel
CombinedCycle_EOL_chromiumsteel = CombinedCycle_chromiumsteel
CombinedCycle_EOL_stonewool = CombinedCycle_stonewool

## 3.8 Module C4

In [18]:
CombinedCycle_EOL_landfill_aluminium = CombinedCycle_EOL_aluminiumCastalloy * LandfillMetals
CombinedCycle_EOL_landfill_aluminium = CombinedCycle_EOL_aluminiumWroughtalloy * LandfillMetals
CombinedCycle_EOL_landfill_ceramictile = CombinedCycle_EOL_ceramictile * LandfillClaybrick
CombinedCycle_EOL_landfill_chromium = CombinedCycle_EOL_chromium * LandfillMetals2
CombinedCycle_EOL_landfill_cobalt = CombinedCycle_EOL_cobalt * LandfillMetals2
CombinedCycle_EOL_landfill_concreteAT = CombinedCycle_EOL_concreteAT * LandfillConcrete
CombinedCycle_EOL_landfill_concreteCH = CombinedCycle_EOL_concreteCH * LandfillConcrete
CombinedCycle_EOL_landfill_copper = CombinedCycle_EOL_copper * LandfillMetals
CombinedCycle_EOL_landfill_nickel = CombinedCycle_EOL_nickel * LandfillMetals2
CombinedCycle_EOL_inc_PEHD = CombinedCycle_EOL_PEHD * IncinerationPEHD
CombinedCycle_EOL_landfill_PEHD = CombinedCycle_EOL_PEHD * LandfillPEHD
CombinedCycle_EOL_landfill_reinforcingsteel = CombinedCycle_EOL_reinforcingsteel * LandfillMetals
CombinedCycle_EOL_landfill_chromiumsteel = CombinedCycle_EOL_chromiumsteel * LandfillMetals2
CombinedCycle_EOL_landfill_stonewool = CombinedCycle_EOL_stonewool * LandfillMineralWool

## 3.9 Module D1 (related to the export of secondary materials)

In [19]:
CombinedCycle_EOL_recycling_aluminiumCastalloy = (RecyclingMetals * RecyclingEfficiencyMetals * CombinedCycle_EOL_aluminiumCastalloy) - (RecycledContentMetals * CombinedCycle_EOL_aluminiumCastalloy)
CombinedCycle_EOL_primary_aluminiumCastalloy = ((RecycledContentMetals * CombinedCycle_EOL_aluminiumCastalloy) 
                                                - (RecyclingMetals * RecyclingEfficiencyMetals * CombinedCycle_EOL_aluminiumCastalloy)) * SRmetals

CombinedCycle_EOL_recycling_aluminiumWroughtalloy = (RecyclingMetals * RecyclingEfficiencyMetals * CombinedCycle_EOL_aluminiumWroughtalloy) - (RecycledContentMetals * CombinedCycle_EOL_aluminiumWroughtalloy)
CombinedCycle_EOL_primary_aluminiumWroughtalloy = ((RecycledContentMetals * CombinedCycle_EOL_aluminiumWroughtalloy) - (RecyclingMetals * RecyclingEfficiencyMetals * CombinedCycle_EOL_aluminiumWroughtalloy)) * SRmetals

CombinedCycle_EOL_recycling_chromium = (RecyclingMetals * RecyclingEfficiencyMetals * CombinedCycle_EOL_chromium) - (RecycledContentMetals * CombinedCycle_EOL_chromium)
CombinedCycle_EOL_primary_chromium = ((RecycledContentMetals * CombinedCycle_EOL_chromium) - (RecyclingMetals * RecyclingEfficiencyMetals * CombinedCycle_EOL_chromium)) * SRmetals

CombinedCycle_EOL_recycling_cobalt = (RecyclingMetals * RecyclingEfficiencyMetals * CombinedCycle_EOL_cobalt) - (RecycledContentMetals * CombinedCycle_EOL_cobalt)
CombinedCycle_EOL_primary_cobalt = ((RecycledContentMetals * CombinedCycle_EOL_cobalt) - (RecyclingMetals * RecyclingEfficiencyMetals * CombinedCycle_EOL_cobalt)) * SRmetals

CombinedCycle_EOL_recycling_concreteAT = (RecyclingConcrete * RecyclingEfficiencyConcrete * CombinedCycle_EOL_concreteAT) - (RecycledContentConcrete * CombinedCycle_EOL_concreteAT)
CombinedCycle_EOL_primary_concreteAT = ((RecycledContentConcrete * CombinedCycle_EOL_concreteAT) - (RecyclingConcrete * RecyclingEfficiencyConcrete * CombinedCycle_EOL_concreteAT)) * SRinert

CombinedCycle_EOL_recycling_concreteCH = (RecyclingConcrete * RecyclingEfficiencyConcrete * CombinedCycle_EOL_concreteCH) - (RecycledContentConcrete * CombinedCycle_EOL_concreteCH)
CombinedCycle_EOL_primary_concreteCH = ((RecycledContentConcrete * CombinedCycle_EOL_concreteCH) - (RecyclingConcrete * RecyclingEfficiencyConcrete * CombinedCycle_EOL_concreteCH)) * SRinert

CombinedCycle_EOL_recycling_copper = (RecyclingMetals * RecyclingEfficiencyMetals * CombinedCycle_EOL_copper) - (RecycledContentMetals * CombinedCycle_EOL_copper)
CombinedCycle_EOL_primary_copper = ((RecycledContentMetals * CombinedCycle_EOL_copper) - (RecyclingMetals * RecyclingEfficiencyMetals * CombinedCycle_EOL_copper)) * SRmetals

CombinedCycle_EOL_recycling_nickel = (RecyclingMetals * RecyclingEfficiencyMetals * CombinedCycle_EOL_nickel) - (RecycledContentMetals * CombinedCycle_EOL_nickel)
CombinedCycle_EOL_primary_nickel = ((RecycledContentMetals * CombinedCycle_EOL_nickel) - (RecyclingMetals * RecyclingEfficiencyMetals * CombinedCycle_EOL_nickel)) * SRmetals

CombinedCycle_EOL_recycling_HDPE = (RecyclingPEHD * RecyclingEfficiencyPlastics * CombinedCycle_EOL_PEHD) - (RecycledContentPlastic * CombinedCycle_EOL_PEHD)
CombinedCycle_EOL_primary_HDPE = ((RecycledContentPlastic * RecyclingEfficiencyPlastics * CombinedCycle_EOL_PEHD) - (RecyclingPEHD * RecyclingEfficiencyPlastics * CombinedCycle_EOL_PEHD)) * SRinert

CombinedCycle_EOL_recycling_reinforcingsteel = (RecyclingMetals * RecyclingEfficiencyMetals * CombinedCycle_EOL_reinforcingsteel) - (RecycledContentMetals * CombinedCycle_EOL_reinforcingsteel)
CombinedCycle_EOL_primary_reinforcingsteel = ((RecycledContentMetals * CombinedCycle_EOL_reinforcingsteel) - (RecyclingMetals * RecyclingEfficiencyMetals * CombinedCycle_EOL_reinforcingsteel)) * SRmetals

CombinedCycle_EOL_recycling_chromiumsteel = (RecyclingMetals * RecyclingEfficiencyMetals * CombinedCycle_EOL_chromiumsteel) - (RecycledContentMetals * CombinedCycle_EOL_chromiumsteel)
CombinedCycle_EOL_primary_chromiumsteel = ((RecycledContentMetals * CombinedCycle_EOL_chromiumsteel) - (RecyclingMetals * RecyclingEfficiencyMetals * CombinedCycle_EOL_chromiumsteel)) * SRmetals


## 3.10 Module D3 (related to the export of energy as result of waste incineration)

In [20]:
CombinedCycle_EOL_energyrecoveryheat_PEHD = - CombinedCycle_EOL_inc_PEHD * LHVHDPE * Xincheat
CombinedCycle_EOL_energyrecoveryele_PEHD = - CombinedCycle_EOL_inc_PEHD * LHVHDPE * Xincele

# 4. Data for export

In [21]:
ThermalEnergyProduced_CombinedCycle = np.array(TotQ)